In [1]:
%matplotlib inline
import rasterio
import matplotlib.pyplot as plt
import gdal
import numpy as np
import timeit

In [2]:
# Function list
def band_average(raster, bands, icethickness):
    band_average = np.zeros((bands.max()+1, 1))
    for n in range(bands.max()+1):
        raster_single = raster.copy()
        raster_single[bands != n] = np.nan
        raster_single[(icethickness == 0) & (emergence_velocity == 0)] = np.nan
        band_average[n,0] = np.nanmean(raster_single)
    return band_average
    

def emergence_pixels(vel_x_raw, vel_y_raw, vel_total, vel_min, vel_depth_avg_factor, icethickness, bands):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    vel_y = vel_y_raw * -1 * vel_depth_avg_factor
    vel_x = vel_x_raw * vel_depth_avg_factor
    # Compute the initial volume
    volume_initial = icethickness * pixel_size**2
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pixel_size)
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    if option_minvelocity == 1:
        vel_x[vel_total < vel_min] = 0
        vel_y[vel_total < vel_min] = 0
    if option_maxvelocity == 1:
        vel_x[vel_total > max_velocity] = 0
        vel_y[vel_total > max_velocity] = 0
    # Remove clusters of high velocity on stagnant portions of glaciers due to feature tracking of ice cliffs and ponds
    if option_stagnantbands == 1:
        vel_x[bands <= stagnant_band] = 0
        vel_y[bands <= stagnant_band] = 0        
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / pixel_size
    vel_y_pix = vel_y / pixel_size
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
#     # Check that mass is conserved (threshold = 0.1 m x pixel_size**2)
#     print('Mass is conserved?', (volume_final.sum() - volume_initial.sum()) < 0.1 * pixel_size**2)
#     print(volume_final.sum() - volume_initial.sum())
    # Final ice thickness
    icethickness_final = volume_final / pixel_size**2
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity


def emergence_statistics(mc_emergence):
    """
    Calculate statistics related to emergence velocities
    Each row is the Monte Carlo simulations of a band
    Each column is a single simulation
    """
    # 1st and 3rd quartiles for box-and-whiskers plots
    quartile_low = 0.25
    quartile_high = 0.75
    # 95% confidence intervals
    confidence_low = 0.025
    confidence_high = 0.975
    # Number of simulations is the number of columns
    sims = mc_emergence.shape[1]
    # Sort the values for each band (row)
    emergence_sorted = np.sort(mc_emergence, axis=1)
    # Output table
    emergence_stats = np.zeros((mc_emergence.shape[0],11+sims))
    # Col 0 = bands
    emergence_stats[:,0] = np.arange(0,mc_emergence.shape[0])
    # Col 1 = median
    emergence_stats[:,1] = np.median(emergence_sorted, axis=1)
    # Col 2 = median absolute deviation
    emergence_stats[:,2] = 1.483 * np.median(
        np.absolute(np.transpose(np.transpose(emergence_sorted) - np.median(emergence_sorted, axis=1))), axis=1)
    # Col 3 = mean
    emergence_stats[:,3] = np.mean(emergence_sorted, axis=1)
    # Col 4 = stdev
    emergence_stats[:,4] = np.std(emergence_sorted, axis=1)
    # Col 5 = min
    emergence_stats[:,5] = np.min(emergence_sorted, axis=1)
    # Col 6 = max
    emergence_stats[:,6] = np.max(emergence_sorted, axis=1)
    # Col 7 = 1st Quartile
    emergence_stats[:,7] = emergence_sorted[:,np.around(quartile_low*sims).astype(int)-1]
    # Col 8 = 3rd Quartile
    emergence_stats[:,8] = emergence_sorted[:,np.around(quartile_high*sims).astype(int)-1]
    # Col 9 = 95% low
    emergence_stats[:,9] = emergence_sorted[:,np.around(confidence_low*sims).astype(int)-1]
    # Col 10 = 95% high
    emergence_stats[:,10] = emergence_sorted[:,np.around(confidence_high*sims).astype(int)-1]
    # Col 11+ = MC simulations
    emergence_stats[:,11:mc_bands_emergence.shape[1]+11] = mc_bands_emergence
    return emergence_stats


def export_raster(raster, output_filename, ds, NoData_value):
    driver = gdal.GetDriverByName('GTiff')
    new_dataset = driver.Create(output_filename,
                                ds.RasterXSize,    # number of columns
                                ds.RasterYSize,    # number of rows
                                1,                  # number of bands
                                gdal.GDT_Float32)  # datatype of the raster
    new_dataset.SetProjection(ds.GetProjection())
    new_dataset.SetGeoTransform(ds.GetGeoTransform())
    # Now we need to set the band's nodata value to -1
    new_band = new_dataset.GetRasterBand(1)
    new_band.SetNoDataValue(NoData_value)
    # And finally, let's write our NDVI array.
    new_band.WriteArray(raster)
    

def import_raster(raster_fn):
    """Open raster and obtain the values in its first band as an array
    Output: array of raster values
    """
    # open raster dataset
    raster_ds = gdal.Open(raster_fn)
    # extract band information and get values
    raster_band = raster_ds.GetRasterBand(1)
    raster_values = raster_band.ReadAsArray()
    return raster_values

In [3]:
# ----- Input for Khumbu Glacier ----------
# Filenames (fn)
vel_x_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_20132016_vx_med_ClipV2.tif'
vel_y_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_20132016_vy_med_ClipV2.tif'
vel_x_std_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_dshean_vx_std_ClipV2.tif'
vel_y_std_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_dshean_vy_std_ClipV2.tif'
icethickness_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_icethickness_Huss.tif'
bands_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_600m_bands_polygons_raster_ClipV2.tif'
# Additional input
mc_simulations = 1000
pixel_size = 30
offglacier_velocity_mean = 4.055
offglacier_velocity_std = 2.587
max_velocity = 200
icethickness_uncertainty_percentage = -21
#  Farinotti et al. [2017] showed on average Huss and Farinotti [2012] underestimate ice thickness by 21%
vel_depth_avg_factor_min = 0.8
vel_depth_avg_factor_max = 0.8
#  Azam et al. (2012) uses 0.9 (state ice flux is +/- 10% based on unknown sliding and other sources of error)
#  Nuimura et al. (2011) uses 0.8 based on Paterson (1994) and Sakai et al. (2006)
#  Vincent et al. (2016) uses 0.8
stagnant_band = 7
output_filename = '../Output/emergence_Khumbu_mcstat_MAD'

# ----- Quality Control Options ----------
# Filter min/max velocity
option_minvelocity = 1
option_maxvelocity = 1
#  > 1 (default) - apply limits specified in model input
#  > 0 - do not apply limits
option_border = 1
#  > 1 (default) - apply the border to prevent errors
#  > 0 - do not apply border, assumes errors near edges are avoided elsewhere
option_stagnantbands = 1
#  > 1 (default) - remove velocities from stagnant bands that are caused by cliffs and ponds
#  > 0 - do not remove the clusters of higher velocities on the tongues of the glaciers

In [4]:
# Open raster for properties (will be used to export tif)
ds = gdal.Open(vel_x_fn)
# Import rasters
vel_x = import_raster(vel_x_fn)
vel_y = import_raster(vel_y_fn)
# try:
#     vel_x_std = import_raster(vel_x_std_fn)
#     vel_y_std = import_raster(vel_y_std_fn)
# else:
vel_x_std = np.zeros((vel_x.shape[0], vel_x.shape[1])) + (offglacier_velocity_std**2/2)**0.5
vel_y_std = np.zeros((vel_x.shape[0], vel_x.shape[1])) + (offglacier_velocity_std**2/2)**0.5
icethickness = import_raster(icethickness_fn)
bands = import_raster(bands_fn)

In [5]:
# Compute the total velocity
vel_total_print = (vel_x**2 + vel_y**2)**0.5
export_raster(vel_total_print,'Imja_dshean_total_vel_20171204.tif', ds, 0)

In [1]:
# MONTE CARLO SIMULATIONS OF EMERGENCE VELOCITY
# Generate normal random numbers (mean 0, stdev 1) to include vx and vy uncertainties (actual values calculated in the)
mc_vm_uncertainty = np.random.normal(0, 1, (1, mc_simulations))
# Generate normal distribution for minimum velocity threshold
mc_vmin_uncertainty = np.random.normal(offglacier_velocity_mean, offglacier_velocity_std, (1, mc_simulations))
# Generate uniform distribution for ice thickness uncertainty between Huss and -21% underestimation
mc_icethickness_uncertainty = np.random.uniform(icethickness_uncertainty_percentage, 0, (1, mc_simulations))
# Generate uniform distribution for ratio of surface to mean velocity
mc_vsfc_ratio_uncertainty = np.random.uniform(vel_depth_avg_factor_min, vel_depth_avg_factor_max, (1, mc_simulations))
# Run the Monte Carlo simulations
for sim in range(mc_simulations):  
    # Print update every x simulations
    if sim % 50 == 0:
        print('Simulation:', sim)
    # Compute velocities at each pixel according to uncertainty
    mc_vel_x = vel_x + (mc_vm_uncertainty[0, sim] * vel_x_std)
    mc_vel_y = vel_y + (mc_vm_uncertainty[0, sim] * vel_y_std)
    # Compute the total velocity
    mc_vel_total = (mc_vel_x**2 + mc_vel_y**2)**0.5
    # Minimum velocity including uncertainty
    mc_vmin = mc_vmin_uncertainty[0, sim]
    # Compute ice thicknesses based on uncertainty
    mc_icethickness = icethickness / (1 + mc_icethickness_uncertainty[0, sim]/100)
    #  percent_uncertainty = (x2 - x1) / x1 * 100
    #  percent_uncertainty is the uncertainty (mean = -21%, stdev = 60%)
    #  x2 is the modeled ice thickness (Huss and Farinotti, 2012)
    #  We want to solve for x1, i.e., the ice thickness with the error included
    #    x1 = x2 / (1 + percent_uncertainty / 100)
    mc_vsfc_ratio = mc_vsfc_ratio_uncertainty[0, sim]
    # Emergence Velocity Calculations for each pixel
    emergence_velocity = emergence_pixels(mc_vel_x, mc_vel_y, mc_vel_total, mc_vmin, mc_vsfc_ratio, mc_icethickness, bands)
    # Emergence Velocity Calculations for each band (average the pixel values)
    bands_emergence = band_average(emergence_velocity, bands, icethickness)
    # Record the results
    if sim == 0:
        mc_bands_emergence = bands_emergence
    else:
        mc_bands_emergence = np.concatenate((mc_bands_emergence, bands_emergence), axis=1)

# Calculate statistics of Monte Carlo simulatuions
mc_bands_emergence_wstats = emergence_statistics(mc_bands_emergence)
# Record uncertainties
uncertainty_table = np.zeros((4, mc_simulations))
# Row 0 = uncertainty wrt surface velocity magnitude
uncertainty_table[0,:] = mc_vm_uncertainty
# Row 1 = uncertainty wrt minimum detectable velocity
uncertainty_table[1,:] = mc_vmin_uncertainty
# Row 2 = uncertainty wrt ice thickness (percentage)
uncertainty_table[2,:] = mc_icethickness_uncertainty
# Row 3 = uncertainty wrt velocity depth average factor
uncertainty_table[3,:] = mc_vsfc_ratio_uncertainty

# Record output of statistics and the uncertainty stats
np.savetxt(output_filename + '.txt', mc_bands_emergence_wstats)
np.savetxt(output_filename + 'uncertaintytable.txt', uncertainty_table)

print('finished')

In [8]:
# plt.figure(1)
# plt.imshow(vel_x)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('x velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(2)
# plt.imshow(vel_y)
# plt.title('y velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(3)
# plt.imshow(icethickness)
# plt.title('ice thickness (m)')
# plt.colorbar()

# vel_total = total_velocity(vel_x, vel_y)
# plt.figure(4)
# plt.imshow(vel_total)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('total velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()